In [2]:
import pandas as pd
iris=pd.read_csv('./input_data/iris.csv')
iris['data_version']=1.0
#iris2=iris.reset_index()
iris.head()

SepalLength  SepalWidth  PetalLength  PetalWidth         Name  data_version
0          5.1         3.5          1.4         0.2  Iris-setosa           1.0
1          4.9         3.0          1.4         0.2  Iris-setosa           1.0
2          4.7         3.2          1.3         0.2  Iris-setosa           1.0
3          4.6         3.1          1.5         0.2  Iris-setosa           1.0
4          5.0         3.6          1.4         0.2  Iris-setosa           1.0

In [1]:
from sqlalchemy import create_engine
import os
pgconfig={
    'host':os.environ['DB_HOST'],
    'port':os.environ['DB_PORT'],
    'database':os.environ['DB_NAME'],
    'user':os.environ['DB_USER'],
    'password':os.environ['DB_PASSWORD']
}
engine=create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig))

In [21]:
columns=['data_version',
       'SepalLength',
       'SepalWidth',
       'PetalLength',
       'PetalWidth']
#iris[columns].head()
iris[columns].to_sql('iris_input',engine,if_exists='append',index=False)

In [2]:
from sqlalchemy import create_engine
import os
pgconfig={
    'host':os.environ['DB_HOST'],
    'port':os.environ['DB_PORT'],
    'database':os.environ['DB_NAME'],
    'user':os.environ['DB_USER'],
    'password':os.environ['DB_PASSWORD']
}
%reload_ext sql
#dsl='postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
#dsl="postgresql://postgres:postgres@db:5432/analysis"
%sql postgresql://postgres:postgres@db:5432/analysis
#%sql postgresql://{user}:{password}@{host}/{database}.format(**pgconfig)

'Connected: postgres@analysis'

In [3]:
%config SqlMagic.autopandas=True
df_input=%sql select * from iris_input
df_input.head()

 * postgresql://postgres:***@db:5432/analysis
150 rows affected.


data_version  SepalLength  SepalWidth  PetalLength  PetalWidth
0           1.0          5.1         3.5          1.4         0.2
1           1.0          4.9         3.0          1.4         0.2
2           1.0          4.7         3.2          1.3         0.2
3           1.0          4.6         3.1          1.5         0.2
4           1.0          5.0         3.6          1.4         0.2

In [32]:
import pandas as pd
from sklearn.cluster import KMeans

df_src=df_input[['SepalLength','SepalWidth','PetalLength','PetalWidth']]

kms=KMeans(n_clusters=3,random_state=1)
clusters=kms.fit_predict(df_src)

df_src['clusters']=clusters
serial_num = pd.RangeIndex(start=1, stop=len(df_src.index) + 1, step=1)
df_src['ID'] = serial_num
df_src.set_index('ID')
columns=['ID',
       'SepalLength',
       'SepalWidth',
       'PetalLength',
       'PetalWidth',
       'clusters']
#df_src.head()
df_src[columns].to_sql('iris_output',engine,if_exists='append',index=False)


In [33]:
%config SqlMagic.autopandas=True
df_output=%sql select * from iris_output
df_output.head()

 * postgresql://postgres:***@db:5432/analysis
150 rows affected.


ID  SepalLength  SepalWidth  PetalLength  PetalWidth  clusters
0   1          5.1         3.5          1.4         0.2         1
1   2          4.9         3.0          1.4         0.2         1
2   3          4.7         3.2          1.3         0.2         1
3   4          4.6         3.1          1.5         0.2         1
4   5          5.0         3.6          1.4         0.2         1

SyntaxError: invalid syntax (<ipython-input-28-aadba3781180>, line 7)

In [5]:
from sklearn.cluster import KMeans
kms=KMeans(n_clusters=3,random_state=1)
clusters=kms.fit_predict(df_src)
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2], dtype=int32)

In [8]:
import joblib
joblib.dump(clusters,'study.pkl')

['study.pkl']

In [6]:
df_src['clusters']=clusters

In [21]:
import pandas as pd
serial_num = pd.RangeIndex(start=1, stop=len(df_src.index) + 1, step=1)
df_src['ID'] = serial_num
df_src.set_index('ID')
df_src.drop('No', axis=1)
df_src.head()

SepalLength  SepalWidth  PetalLength  PetalLength  clusters  No  ID
0          5.1         3.5          1.4          1.4         0   1   1
1          4.9         3.0          1.4          1.4         0   2   2
2          4.7         3.2          1.3          1.3         0   3   3
3          4.6         3.1          1.5          1.5         0   4   4
4          5.0         3.6          1.4          1.4         0   5   5

In [15]:
df_src.head()

SepalLength  SepalWidth  PetalLength  PetalLength  clusters  No
0          5.1         3.5          1.4          1.4         0   1
1          4.9         3.0          1.4          1.4         0   2
2          4.7         3.2          1.3          1.3         0   3
3          4.6         3.1          1.5          1.5         0   4
4          5.0         3.6          1.4          1.4         0   5

In [7]:
df_src.head(100)

SepalLength  SepalWidth  PetalLength  PetalLength  clusters
0           5.1         3.5          1.4          1.4         0
1           4.9         3.0          1.4          1.4         0
2           4.7         3.2          1.3          1.3         0
3           4.6         3.1          1.5          1.5         0
4           5.0         3.6          1.4          1.4         0
5           5.4         3.9          1.7          1.7         0
6           4.6         3.4          1.4          1.4         0
7           5.0         3.4          1.5          1.5         0
8           4.4         2.9          1.4          1.4         0
9           4.9         3.1          1.5          1.5         0
10          5.4         3.7          1.5          1.5         0
11          4.8         3.4          1.6          1.6         0
12          4.8         3.0          1.4          1.4         0
13          4.3         3.0          1.1          1.1         0
14          5.8         4.0          1.2          1.2         0
15          5.7         4.4          1.5          1.5         0
16          5.4         3.9          1.3          1.3         0
17          5.1         3.5          1.4          1.4         0
18          5.7         3.8          1.7          1.7         0
19          5.1         3.8          1.5          1.5         0
20          5.4         3.4          1.7          1.7         0
21          5.1         3.7          1.5          1.5         0
22          4.6         3.6          1.0          1.0         0
23          5.1         3.3          1.7          1.7         0
24          4.8         3.4          1.9          1.9         0
25          5.0         3.0          1.6          1.6         0
26          5.0         3.4          1.6          1.6         0
27          5.2         3.5          1.5          1.5         0
28          5.2         3.4          1.4          1.4         0
29          4.7         3.2          1.6          1.6         0
..          ...         ...          ...          ...       ...
70          5.9         3.2          4.8          4.8         2
71          6.1         2.8          4.0          4.0         2
72          6.3         2.5          4.9          4.9         2
73          6.1         2.8          4.7          4.7         2
74          6.4         2.9          4.3          4.3         2
75          6.6         3.0          4.4          4.4         2
76          6.8         2.8          4.8          4.8         2
77          6.7         3.0          5.0          5.0         1
78          6.0         2.9          4.5          4.5         2
79          5.7         2.6          3.5          3.5         2
80          5.5         2.4          3.8          3.8         2
81          5.5         2.4          3.7          3.7         2
82          5.8         2.7          3.9          3.9         2
83          6.0         2.7          5.1          5.1         2
84          5.4         3.0          4.5          4.5         2
85          6.0         3.4          4.5          4.5         2
86          6.7         3.1          4.7          4.7         2
87          6.3         2.3          4.4          4.4         2
88          5.6         3.0          4.1          4.1         2
89          5.5         2.5          4.0          4.0         2
90          5.5         2.6          4.4          4.4         2
91          6.1         3.0          4.6          4.6         2
92          5.8         2.6          4.0          4.0         2
93          5.0         2.3          3.3          3.3         2
94          5.6         2.7          4.2          4.2         2
95          5.7         3.0          4.2          4.2         2
96          5.7         2.9          4.2          4.2         2
97          6.2         2.9          4.3          4.3         2
98          5.1         2.5          3.0          3.0         2
99          5.7         2.8          4.1          4.1         2

[100 rows x 5 columns]